In [1]:
from transformers import AutoTokenizer,AutoModel
import torch
import torch.cuda as cuda
#加载分词器
tokenizer = AutoTokenizer.from_pretrained('hfl/chinese-roberta-wwm-ext')
#加载预训练模型
pretrained = AutoModel.from_pretrained('hfl/chinese-roberta-wwm-ext')

device = torch.device('cuda' if cuda.is_available() else 'cpu')
# pretrained = pretrained.to(device)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
# 定义下游模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.tuning = False
        self.pretrained = None

        self.rnn = torch.nn.GRU(768, 768, batch_first=True)
        self.fc = torch.nn.Linear(768, 8)

    def forward(self, inputs):
        if self.tuning:
            out = self.pretrained(**inputs).last_hidden_state
        else:
            with torch.no_grad():
                out = pretrained(**inputs).last_hidden_state

        out, _ = self.rnn(out)

        out = self.fc(out).softmax(dim=2)

        return out


In [8]:
model_load = torch.load('ner5.model')
model_load.eval()    
model_load.to('cpu')

def custom_predict(sentence):

    inputs = tokenizer.encode_plus([sentence],
                                   truncation=True,
                                   padding=True,
                                   return_tensors='pt',
                                   is_split_into_words=True)

    with torch.no_grad():
        outputs = model_load(inputs)

    # [b, lens] -> [b, lens]
    preds = outputs.argmax(dim=2)[0]

    result = ''

    res = [set() for _ in range(3)]
    tmp = ''
    current_flag = -1

    for i in range(len(preds)):
        if inputs['attention_mask'][0][i] == 1:
            result += tokenizer.decode(inputs['input_ids'][0][i])+' '
            result += str(preds[i].item())+' '
            num = preds[i].item()
            # num 不为0和7和#表示该词为关键词
            if (num != 0 and num != 7 and num != '#'):
                # 关键词开始为奇数
                if (num & 1):
                    # 将形如 广5东6广5州6 拆成两个词
                    if (len(tmp) > 1):
                        res[(current_flag-1)//2].add(tmp)
                        tmp = ''
                    current_flag = num
                    tmp += tokenizer.decode(inputs['input_ids'][0][i])

                    # 防止形如 X4X4 出现
                elif (num & 1 == 0 and current_flag != -1):
                    tmp += tokenizer.decode(inputs['input_ids'][0][i])
            else:
                if (len(tmp) > 1):
                    # current_flag 1对应姓名下标0，3对应组织下表1，5对应地点下表2
                    res[(current_flag-1)//2].add(tmp)
                    tmp = ''
                    current_flag = -1

    return res

In [9]:
input_sentence = "2016.9-2019.7          广州二中           高中"
output_prediction = custom_predict(input_sentence)
print(
    f'姓名{list(output_prediction[0])},组织{list(output_prediction[1])},地点{list(output_prediction[2])}')


姓名[],组织['广州二中高中'],地点[]
